In [1]:
import torch
import json
import os
import random
import numpy as np
import torch
import torch.multiprocessing as mp
import torch
from torch.utils.data import DataLoader
import copy
from torch import nn
%matplotlib inline
import matplotlib.pyplot as plt
from dnaDataloader import expermentDataloader
from dnaDataloader import addData
from dnaModelUtil import train
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")
batch_size = 25
device

device(type='cuda', index=0)

In [2]:
try:
   mp.set_start_method('spawn')
except RuntimeError:
   pass # throws error if run twice without resetting the kernal, if its already set we dont care that this errors

## Eval with small traing sets
The goal here is to eval the model while training on small sets of data. That is how well can MLP learn from 100-200 examples of single-molecule experiments.
A verity of samples have been provided anf preprocessed (converted from excel to csv and had empty frames added) to the folder <pre>/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted</pre>

In [3]:
folders = [d[0] for d in os.walk("/users/kent/student/khood5/GitHub/SNN-DNA-project/Prepocessing/sorted")][1:] # remove first one is it is "/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted"
len(folders)

6

In [4]:
datasets = []
featIn = 0
for d in folders:
    data = expermentDataloader(
        f"{d}/index.csv",
        f"{d}", 
    )
    rawData = [d for d in data]
    featIn = len(rawData[0][0])
    trainValidData = []
    testData = []
    addData(testData, trainValidData, rawData, rhsSize=300)


    np.random.shuffle(trainValidData)
    trainData = []
    validData = []
    addData(trainData, validData, trainValidData, rhsSize=int(len(trainValidData)*(1/3)))

    datasets.append({"name": f"{os.path.basename(d)}", 
                     "train":DataLoader(trainData, batch_size=batch_size, shuffle=True) , 
                     "valid":DataLoader(validData, batch_size=batch_size, shuffle=True) , 
                     "test":DataLoader(testData, batch_size=len(testData), shuffle=True) ,
                     "model": {}}
                    )

In [5]:
print(f"datasets:")
for d in datasets:
    print(d)
print(f"{len(datasets)}")
print(f"featIn: {featIn}")

datasets:
{'name': '50_nM_AR_600', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f995a4ffc70>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f995a4ffbe0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f9aa8b8a500>, 'model': {}}
{'name': '1200_nM_AR_600', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f9aa8be0760>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f995a4e53c0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f995a4e5780>, 'model': {}}
{'name': '800_nM_AR_600', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f995a53c1f0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f995a523c70>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f995a584250>, 'model': {}}
{'name': '1800_nM_AR_600', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f995a5843d0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f995a53de70>, 'test': <torc

In [6]:
for d in datasets:
    print(f"-- {d['name']} --")
    print(f"train: {len(d['train'])}")
    print(f"valid: {len(d['valid'])}")
    print(f"test : {len(d['test'])}")

-- 50_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 1200_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 800_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 1800_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 100_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 400_nM_AR_600 --
train: 8
valid: 4
test : 1


In [7]:
manager = mp.Manager()
return_dict = manager.dict()

In [8]:
processes = []
devices = [torch.device("cuda:0")]
epochs = 10000
for error_margin in range(10,21):
    for d in datasets:
        processes.append(mp.Process(target=train, args=(d["train"], d["valid"], f"{d['name']}_{error_margin}", featIn, return_dict, epochs, error_margin, devices[0])))
        devices.append(devices.pop(0))
    
processes

[<Process name='Process-2' parent=3469115 initial>,
 <Process name='Process-3' parent=3469115 initial>,
 <Process name='Process-4' parent=3469115 initial>,
 <Process name='Process-5' parent=3469115 initial>,
 <Process name='Process-6' parent=3469115 initial>,
 <Process name='Process-7' parent=3469115 initial>,
 <Process name='Process-8' parent=3469115 initial>,
 <Process name='Process-9' parent=3469115 initial>,
 <Process name='Process-10' parent=3469115 initial>,
 <Process name='Process-11' parent=3469115 initial>,
 <Process name='Process-12' parent=3469115 initial>,
 <Process name='Process-13' parent=3469115 initial>,
 <Process name='Process-14' parent=3469115 initial>,
 <Process name='Process-15' parent=3469115 initial>,
 <Process name='Process-16' parent=3469115 initial>,
 <Process name='Process-17' parent=3469115 initial>,
 <Process name='Process-18' parent=3469115 initial>,
 <Process name='Process-19' parent=3469115 initial>,
 <Process name='Process-20' parent=3469115 initial>,
 

In [9]:
print(return_dict)

{}


In [10]:
processesList = list(range(len(processes)))

while processesList:
    run = processesList[:1]
    processesList = processesList[1:]
    for i in run:
        processes[i].start()
    for i in run:
        processes[i].join()
        processes[i].terminate()
print(return_dict)

training 50_nM_AR_600_10 on cuda:0...


KeyboardInterrupt: 

In [ ]:
results = dict(return_dict)
with open("./Models/smallTrain/results.json", 'w') as file:
    json_object = json.dumps(results, indent=4)
    file.write(json_object)